# Prerequisites
Need to have IBM ILOG CPLEX Optimization Studio installed locally on your machine.

For students:
https://www.ibm.com/academic/
1. Access sofware downloads
2. Sign in
3. Data Science
4. ILOG CPLEX Optimization Studio
5. Selecy HTTP as Download method

Otherwise:
https://www.ibm.com/products/ilog-cplex-optimization-studio/pricing

%pip install docplex

In [ ]:
# Imports
import os
import json
import numpy as np
import networkx as nx
from math import sqrt
from natsort import natsorted
import docplex
from docplex.mp.model import Model

In [ ]:
def read_nxgraph(filename: str) -> nx.Graph(): # type: ignore
    graph = nx.Graph()
    with open(filename, 'r') as file:
        # lines = []
        line = file.readline()
        is_first_line = True
        while line is not None and line != '':
            if '//' not in line:
                if is_first_line:
                    strings = line.split(" ")
                    num_nodes = int(strings[0])
                    num_edges = int(strings[1])
                    nodes = list(range(num_nodes))
                    graph.add_nodes_from(nodes)
                    is_first_line = False
                else:
                    node1, node2, weight = line.split()
                    graph.add_edge(int(node1), int(node2), weight=weight)
            line = file.readline()
    return graph

In [ ]:
def cplex_maxcut(graph):

    # Create QUBO matrix
    nodes = len(list(graph.nodes))
    J = nx.to_numpy_array(graph)
    
    # Construct model
    model = Model("maxcut_qubo")

    # Set time limit 1 hr
    model.set_time_limit(900)
    model.set_log_output(True) 
    model.parameters.mip.tolerances.mipgap=0.0

    # Create variable for each vertex
    x = model.binary_var_list(nodes)

    # Objective function
    objective = model.sum(
        -J[i, j] * (2 * x[i] - 1) * (2 * x[j] - 1) * (1/sqrt(nodes))
        for i in range(nodes) for j in range(i + 1, nodes) if J[i, j] != 0.0
    )
    model.minimize(objective)
    
    # Solve
    solution = model.solve()
    sol = ""
    for i in range(nodes):
        sol += f"{int(x[i].solution_value)}"
    return float(solution.objective_value), sol
    

In [ ]:
# graph = read_nxgraph("../data/RL/spin_glass/10x10/GSRL_100_ID2.txt")
# obj_val, sol = cplex_maxcut(graph)

In [ ]:
# Parse all files in data_dir 
data_dir = f"../data/vna/SK"
input_files = [ f for f in os.listdir(data_dir) ]
input_files = natsorted(input_files)

# Write directories & files
results_dir = f"../results/{'/'.join(data_dir.split('/')[2:])}"
solutions_dir = f"../solutions/{'/'.join(data_dir.split('/')[2:])}"
os.makedirs(results_dir, exist_ok=True)
os.makedirs(solutions_dir, exist_ok=True)

i = 0
for file in input_files:
    i += 1
    if i > 25: break
    
    graph = read_nxgraph(f'{data_dir}/{file}') 
    obj_val, sol = cplex_maxcut(graph)

    with open(f"{results_dir}/CPLEX.txt", "a") as f:
        f.write(f"{obj_val}\n")

    with open(f"{solutions_dir}/CPLEX.txt", "a") as f:
        f.write(f"{sol}\n")
